清理数据空值

In [14]:
import pandas as pd

# 修改这里：写你的 csv 文件路径
df = pd.read_csv("/Users/simiao/研究生课程/5507/小组作业/最终/bilibili_comments_with labels.csv", encoding="utf-8")

# 删除 content 为空的行
df = df[df['content'].fillna("").str.strip() != ""]

# 保存清洗后的文件
df.to_csv("/Users/simiao/研究生课程/5507/小组作业/最终/bilibili_comments_clean.csv", index=False, encoding="utf-8-sig")

print("清洗完成！已生成 bilibili_comments_clean.csv")


清洗完成！已生成 bilibili_comments_clean.csv


清洗多次换行的内容+清洗文本中@+用户名的内容

In [15]:
import pandas as pd
import re

# 读取文件
df = pd.read_csv("/Users/simiao/研究生课程/5507/小组作业/最终/bilibili_comments_with labels.csv", encoding="utf-8")

# 删除 content 为空的行
df = df[df['content'].fillna("").str.strip() != ""]

# 定义一个清洗函数
def clean_content(text):
    if not isinstance(text, str):
        return ""

    # 1. 删除 @ + 用户名（匹配中文/英文/数字/下划线）
    text = re.sub(r"@\w+|@[\u4e00-\u9fa5_a-zA-Z0-9]+", "", text)

    # 2. 多个换行 → 一个换行
    text = re.sub(r"\n+", "\n", text)

    # 3. 如果你想把换行替换成空格（让内容变成连续句子），用下面这一行：
    text = text.replace("\n", " ")

    # 4. 删除两边多余空格
    text = text.strip()

    return text

# 对 content 列应用清洗函数
df['content'] = df['content'].apply(clean_content)

# 保存清洗后的文件
df.to_csv("/Users/simiao/研究生课程/5507/小组作业/最终/bilibili_comments_with labels_final.csv", index=False, encoding="utf-8-sig")

print("清洗完成！已生成 bilibili_comments_with labels_final.csv")


清洗完成！已生成 bilibili_comments_with labels_final.csv


继续删除二次清理后产生的空值

In [17]:
import pandas as pd

# 你的原始 CSV 路径
file_path = "/Users/simiao/研究生课程/5507/小组作业/最终/bilibili_comments_with labels_final.csv"

# 读取文件
df = pd.read_csv(file_path)

# 删除 content 为空值（NaN 或 空字符串）的行
df = df[df['content'].notna()]              # 删除 NaN
df = df[df['content'].str.strip() != ""]    # 删除空字符串或只有空格的情况

# 覆盖写回原文件
df.to_csv(file_path, index=False)

print("已完成：已从源文件中删除 content 为空的行并覆盖保存。")


已完成：已从源文件中删除 content 为空的行并覆盖保存。


清理只有数字的评论

In [16]:
import pandas as pd
import re

file_path = "/Users/simiao/研究生课程/5507/小组作业/最终/bilibili_comments_with labels_final.csv"

# 读取
df = pd.read_csv(file_path)

def is_only_digits_except_666(text):
    if pd.isna(text):  # 空值交给你的其它规则
        return False
    text = str(text).strip()
    # 如果是“666”则保留
    if text == "666":
        return False
    # 检查是否全是数字（允许带空格或换行）
    return re.fullmatch(r"\s*\d+\s*", text) is not None

# 删除只含数字但不是 666 的行
df = df[~df['content'].apply(is_only_digits_except_666)]

# 覆盖写回原 CSV
df.to_csv(file_path, index=False)

print("已完成：已删除只有数字（保留 666）的 content 行。")



已完成：已删除只有数字（保留 666）的 content 行。


分割

In [13]:
import pandas as pd

# 你的原 CSV 路径
input_file = "/Users/simiao/研究生课程/5507/小组作业/最终/bilibili_comments_cleanover.csv"

# 读取整个 CSV（21 万行）
df = pd.read_csv(input_file, encoding="utf-8")

# 前 10 万行
df[:100000].to_csv("part1_100k.csv", index=False, encoding="utf-8-sig")

# 剩余部分
df[100000:].to_csv("part2_rest.csv", index=False, encoding="utf-8-sig")

print("切分完成！已生成 part1_100k.csv 和 part2_rest.csv")


切分完成！已生成 part1_100k.csv 和 part2_rest.csv


删除纯噪音行

In [19]:
import pandas as pd
import regex as re  # 注意：pip install regex

df = pd.read_csv("/Users/simiao/研究生课程/5507/小组作业/最终/bilibili_comments_with labels_final.csv")  # 替换为你的 CSV 文件名

def is_noise(text):
    if not isinstance(text, str):
        return False
    
    # 去掉首尾空白后判断空行
    if text.strip() == "":
        return True
    
    # 若包含以下任意内容 → 必须保留
    keep_patterns = [
        r"[a-zA-Z]",            # 英文
        r"\d",                  # 数字
        r"\p{Han}",             # 中文
        r"[!?]",                # ? !
        r"\p{Emoji}",           # emoji
    ]
    
    for pat in keep_patterns:
        if re.search(pat, text):
            return False  # 有这些字符 → 保留
    
    # 仅由符号组成（P=标点, S=符号, \s=空白）→ 删除
    if re.fullmatch(r"[\p{P}\p{S}\s]+", text):
        return True
    
    return False

# 假设你的列名为 content
df = df[~df["content"].apply(is_noise)]

# 覆盖原文件
df.to_csv("yourfile.csv", index=False, encoding="utf-8-sig")
